# <span style="color:blue; font-weight:bold;">5강. 실전, 데이터 분석</span>
- 셋째마당 (09장)
- pp.222-275

# 9. 한국복지패널 데이터 분석

In [ ]:
# 그래프 해상도 설정
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi' : '100'})
%config InlineBackend.figure_format = 'retina'

## 09-1. '한국복지패널 데이터' 분석 준비하기 (225-228쪽)

- 한국복지패널데이터
    - 한국보건사회연구원 발간 조사자료. 2006년부터 전국에서 7천여 가구 선정해서 매년 조사
    - 천여 개 변수로 구성(경제활동, 생활실태, 복지욕구 등)

### [Do it! 실습] 데이터 분석 준비하기(225쪽)

In [ ]:
# Koweps_hpwc14_2019_beta2.sav 파일 다운로드 -> 워킹 디렉터리
# 2020년 발간. 6,331가구, 14,418명 정보 

In [ ]:
pip install pyreadstat

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# 데이터 불러오기
raw_welfare = pd.read_spss('Koweps_hpwc14_2019_beta2.sav')

In [ ]:
# 복사본 만들기
welfare = raw_welfare.copy()

In [ ]:
# 데이터 검토
welfare

In [ ]:
welfare.shape

In [ ]:
welfare.info()

In [ ]:
welfare.describe()

In [ ]:
# 열 이름 변경
welfare = welfare.rename(columns = {'h14_g3'     : 'sex',            #  성별
                                    'h14_g4'     : 'birth',          #  태어난 연도
                                    'h14_g10'    : 'marriage_type',  #  혼인 상태
                                    'h14_g11'    : 'religion',       #  종교 
                                    'p1402_8aq1' : 'income',         #  월급 
                                    'h14_eco9'   : 'code_job',       #  직업 코드
                                    'h14_reg7'   : 'code_region'})   #  지역 코드

## 09-2. 성별에 따른 월급 차이 - 성별에 따라 월급이 다를까? (229-234쪽)

### [Do it! 실습] 성별 변수 검토 및 전처리하기(229쪽)

In [ ]:
welfare['sex'].dtypes # 변수 타입 출력

In [ ]:
welfare['sex'].value_counts() # 빈도 구하기

In [ ]:
# 이상치 결측 처리
welfare['sex'] = np.where(welfare['sex'] == 9, np.nan, welfare['sex'])
 
# 결측치 확인
welfare['sex'].isna().sum()

In [ ]:
# 성별 항목 이름 부여
welfare['sex'] = np.where(welfare['sex'] == 1, 'male', 'female')

# 빈도 구하기
welfare['sex'].value_counts()

In [ ]:
# 빈도 막대 그래프 만들기
sns.countplot(data = welfare, x = 'sex')

### [Do it! 실습] 월급 변수 검토 및 전처리하기(231쪽)

In [ ]:
welfare['income'].dtypes

In [ ]:
welfare['income'].describe()

In [ ]:
sns.histplot(data = welfare, x = 'income')

In [ ]:
welfare['income'].isna().sum()

In [ ]:
# 이상치 결측 처리
welfare['income'] = np.where(welfare['income'] == 9999, np.nan, welfare['income'])

# 결측치 확인
welfare['income'].isna().sum()

### [Do it! 실습] 성별에 따른 월급 차이 분석하기(234쪽)

In [ ]:
sex_income = welfare.groupby('sex', as_index = False) \
                    .agg(mean_income = ('income', 'mean'))
sex_income

In [ ]:
sns.barplot(data = sex_income, x = 'sex', y = 'mean_income')

# 10. 텍스트 마이닝

## 10-1 대통령 연설문 텍스트 마이닝 (279-292쪽)

### `KiWi` 패키지 설치하기

In [ ]:
!pip3 install --upgrade pip
!pip3 install kiwipiepy

### 가장 많이 사용된 단어 알아보기

#### 1. 연설문 불러오기

In [ ]:
moon = open('speech_moon.txt', encoding = 'UTF-8').read()
moon

#### 2. 불필요한 문자 제거하기

In [ ]:
# 불필요한 문자 제거하기
import re
moon = re.sub('[^가-힣]', ' ', moon)
moon

#### 3. 명사 추출하기

In [ ]:
from kiwipiepy import Kiwi
kiwi=Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=False)

In [ ]:
text = "대한민국의 영토는 한반도와 그 부속도서로 한다"

In [ ]:
kiwi.tokenize(text)

In [ ]:
# 사용자 사전 등록
kiwi.add_user_word('부속도서', 'NNG')
kiwi.tokenize(text)

In [ ]:
def extract_noun(text):
    result = kiwi.tokenize(text)
    nouns = []  # 명사를 저장할 리스트
    for token in result:
        if token.tag in ['NNG', 'NNP']:
            nouns.append(token.form)  # 명사를 리스트에 추가
    return nouns  # 모든 명사가 포함된 리스트 반환

extract_noun(text)

In [ ]:
# 데이터 프레임으로 변환
nouns = extract_noun(moon)

import pandas as pd
df_word = pd.DataFrame({'word' : nouns})
df_word

#### 4. 단어 빈도표 만들기

In [ ]:
df_word['count'] = df_word['word'].str.len()
# df_word = df_word.assign(count = lambda x: x['word'].str.len())
# df_word = df_word.assign(count = df_word['word'].str.len())
df_word

In [ ]:
# 두 글자 이상 단어만 남기기
df_word = df_word.query('count >= 2') \
                 .sort_values('count')
df_word

In [ ]:
## 단어 빈도 구하기

# 단어별 분리
# 빈도 구하기
# 내림차순 정렬
df_word = df_word.groupby('word', as_index = False) \
                 .agg(n = ('word', 'count')) \
                 .sort_values('n', ascending = False)
df_word

#### 5. 단어 빈도 막대 그래프 만들기

In [ ]:
# 단어 빈도 상위 20개 추출
top20 = df_word.head(20)
top20

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams.update({'font.family'    : 'Malgun Gothic',  # 한글 폰트 설정
                     'figure.dpi'     : '120',            # 해상도 설정
                     'figure.figsize' : [6.5, 6]})        # 가로 세로 크기 설정

# 막대 그래프 만들기
sns.barplot(data = top20, y = 'word', x = 'n')

## 10-2 기사 댓글 텍스트 마이닝 (293-299쪽)

### 가장 많이 사용된 단어 알아보기

#### 1. 기사 댓글 불러오기

In [ ]:
# 데이터 불러오기
df = pd.read_csv('news_comment_BTS.csv', encoding = 'UTF-8')
df

In [ ]:
# 데이터 살펴보기
df.info()

#### 2. 불필요한 문자 제거하기

In [ ]:
# 불필요한 문자 제거하기
df['reply'] = df['reply'].str.replace('[^가-힣]', ' ', regex = True)
df['reply'].head()

#### 3. 명사 추출하기

In [ ]:
# 명사 추출 - apply() 활용
nouns = df['reply'].apply(lambda x: extract_noun(x))
nouns

#### 4. 단어 빈도표 만들기

In [ ]:
# 한 행에 한 단어가 들어가도록 구성
nouns = nouns.explode()
nouns

In [ ]:
### 이 결과를 원상태로 복구시키기_시작 ###

In [ ]:
# 시리즈 -> df로 전환하고, 복제된 인덱스 번호를 열로 전환

## df로 전환
temp = nouns.to_frame()
temp

In [ ]:
# 인덱스 번호를 열로 전환
temp.index.name = 'sent'
temp = temp.reset_index()
temp

In [ ]:
# groupby()와 agg() 이용해서 복원
temp.groupby('sent') \
    ['reply'] \
    .agg(list)

In [ ]:
### 이 결과를 원상태로 복구시키기_끝 ###

In [ ]:
# 데이터 프레임 만들기
df_word = pd.DataFrame({'word' : nouns})

# 글자 수 추가
df_word['count'] = df_word['word'].str.len()

# 두 글자 이상 단어만 남기기
df_word = df_word.query('count >= 2')
df_word

In [ ]:
## 빈도표 만들기

# 단어별 분리
# 빈도 구하기
# 내림차순 정렬
df_word = df_word.groupby('word', as_index = False) \
                 .agg(n = ('word', 'count')) \
                 .sort_values('n', ascending = False)
df_word

#### 5. 단어 빈도 막대 그래프 만들기

In [ ]:
# 단어 빈도 상위 20개 추출
top20 = df_word.head(20)
top20

In [ ]:
# 가로 세로 크기 설정
plt.rcParams.update({'figure.figsize': [6.5, 6]})

# 막대 그래프 만들기
sns.barplot(data = top20, y = 'word', x = 'n')

# 11. 지도 시각화

## 11-1 시군구별 인구 단계 구분도 만들기 (301-307쪽)

### 시군구별 인구 단계 구분도 만들기

#### 1. 시군구 경계 지도 데이터 준비하기

In [ ]:
import json
geo = json.load(open('SIG.geojson', encoding = 'UTF-8'))

In [ ]:
len(geo)

In [ ]:
iterator = iter(geo.items())
for _ in range(3):  # 상위 3개의 키-값 쌍만 추출
    print(next(iterator, None))

In [ ]:
# 행정 구역 코드 출력 
geo['features'][0]['properties']

In [ ]:
# 위도, 경도 좌표 출력
geo['features'][0]['geometry']

#### 2. 시군구별 인구 데이터 준비하기

In [ ]:
import pandas as pd
df_pop = pd.read_csv('Population_SIG.csv')
df_pop.head()

In [ ]:
df_pop.info()

In [ ]:
# 행정 구역 코드를 문자 타입으로 변환
df_pop['code'] = df_pop['code'].astype(str)

#### 3. 단계 구분도 만들기

In [ ]:
pip install folium

**(1) 배경 지도 만들기**

In [ ]:
import folium
folium.Map(location = [35.95, 127.7],  # 지도 중심 좌표
           zoom_start = 8)             # 확대 단계

In [ ]:
map_sig = folium.Map(location = [35.95, 127.7],  # 지도 중심 좌표
                     zoom_start = 8,             # 확대 단계
                     tiles = 'cartodbpositron')  # 지도 종류
map_sig

**(2) 단계 구분도 만들기**

In [ ]:
# 지도 데이터
# 통계 데이터
# df_pop 행정 구역 코드, 인구
# geo 행정 구역 코드
folium.Choropleth(geo_data = geo,
                  data = df_pop,
                  columns = ('code', 'pop'),
                  key_on = 'feature.properties.SIG_CD') \
      .add_to(map_sig)

map_sig

**(3) 계급 구간 정하기**

In [ ]:
bins = list(df_pop['pop'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
bins

**(4) 디자인 수정하기**

In [ ]:
## 배경 지도 만들기

# 지도 중심 좌표
# 확대 단계
# 지도 종류
map_sig = folium.Map(location = [35.95, 127.7],
                     zoom_start = 8,
                     tiles = 'cartodbpositron')

In [ ]:
## 단계 구분도 만들기

# 지도 데이터
# 통계 데이터
# df_pop 행정 구역 코드, 인구
# geo 행정 구역 코드
# 컬러맵
# 투명도
# 경계선 투명도
# 계급 구간 기준값
# 배경 지도에 추가
folium.Choropleth(geo_data = geo,
                  data = df_pop,
                  columns = ('code', 'pop'),
                  key_on = 'feature.properties.SIG_CD',
                  fill_color = 'YlGnBu',
                  fill_opacity = 1,
                  line_opacity = 0.5,
                  bins = bins) \
      .add_to(map_sig)

map_sig